# Convert quantum results to GPT3 strings

- Count total shots
- Filter errors
- Convert binary string results to places
- Convert probabilities to GPT3 strings

In [41]:
# JSON data
json_data = {
    "computer": "ionq",  # quantum computer to use
    "qbit_count": 6,     # number of qubits to use
    "places": [          # list of places to visit and their probabilities
        {"place": "Gym", "probability": 0.2},
        {"place": "Opera", "probability": 0.5},
        {"place": "Rooftop bar", "probability": 0.9},
        {"place": "Street fair", "probability": 0.0},
        {"place": "Pool", "probability": 0.1},
    ]
}

# Quantum computer results
result = {
  "10000": 85,
  "00001": 832,
  "00010": 99,
  "00100": 4,
  "01000": 1,
  "01100": 2,
  "00000": 1,
}

In [42]:
# Total shots
total_shots = sum(result.values())
print(total_shots)

1024


In [43]:
# Detect errors and remove it from the results
def filter_errors(result):
    return {key: value for key, value in result.items() if key.count('1') == 1}

# Clean results
cleaned_result = filter_errors(result)
print(cleaned_result)

{'10000': 85, '00001': 832, '00010': 99, '00100': 4, '01000': 1}


In [48]:
# Convert binary strings to places
def convert_binary_to_place(binary_string, json_data):
    index = binary_string.index('1')
    return json_data['places'][index]['place']

place_string = convert_binary_to_place('00001', json_data)
print(place_string)

Pool


In [49]:
# Convert shots to probabilities
def convert_shots_to_probabilities(result, total_shots):
    return {key: value / total_shots for key, value in result.items()}

processed_results = convert_shots_to_probabilities(result, total_shots)
print(processed_results)

{'10000': 0.0830078125, '00001': 0.8125, '00010': 0.0966796875, '00100': 0.00390625, '01000': 0.0009765625, '01100': 0.001953125, '00000': 0.0009765625}
